<h1><center>Intuitive Explanation of Non-stationary Gaussian Process Kernels</center></h1>

With the Tokyo Olympics coming to an end, this seems to be a great time to look at how the athletes have performed over the years. Particularly, let us take a look at the pace of Marathon Gold Medal winners since 1896.

                  [Image - Stationary | Image - Nonstationary] \
Which of the two images do you think better describe the expected trend of the runners' pace over the years?

The right plot which looks a better fit for this data is made using something known as a *non-stationary kernel* in Gaussian processes.


## Background

Let us first look at Gaussian Processes, which we use to predict these trends.


### What are GPs

Gaussian Processes (GPs) are really powerful learning methods, designed to solve classification and regression problems. Along with giving the predictions, they are even able to estimate the uncertainty of the output predictions. This allows us to be mindful of the confidence with which we can accept/reject a predicted output.


Görtler, et al. provide an excellent <a style="text-decoration:none" href="https://distill.pub/2019/visual-exploration-gaussian-processes/"> visual exploration of Gaussian Processes </a> with mathematical intuition as well as a deeper understanding of how they actually work. In this article, we will go over some shortcomings of standard GPs and talk about the extensions that can be built upon them.

### Kernels

If you didn’t notice, Gaussian Processes are named after Gaussian distributions, that are its basic building blocks. If we talk about regression problems, i.e., finding a function y = f(X) that can best describe the data X, we are fairly dealing with multivariate gaussian distributions.

The multivariate Gaussian distribution is defined by a mean vector μ and a covariance matrix Σ.

<center>
$X = \begin{bmatrix}
X_{1}\\ 
X_{2}\\ 
.\\ 
.\\ 
.\\ 
X_{n}
\end{bmatrix} \sim N(\mu , \sum)$
</center>

The mean vector would give you the expected value, like any other regression model. Therefore, the covariance matrix makes the core of the GP regression models. The covariance functions (also called the kernels) involved in Σ describe the joint variability of the Gaussian process random variables.




<center>
<img src="https://images.unsplash.com/photo-1540479859555-17af45c78602?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=750&q=80" width="400">
</center>


Lets us look at an analogy to covariance in kernel functions:

Assume we have a few children standing side-by-side in a line; and we ask them to move randomly back and forth. We’ll see that each child could be standing wherever he/she desires, unaffected by other children. However, if we do the same experiment with each child, holding the hands of the neighbouring child tightly; it would be observed that the neighbouring children would be standing quite closer to each other in the final configuration.

The initial set up for the experiment is analogous to a low correlation coefficient while the latter resembles a high correlation coefficient. The kernel function in GP regression is responsible for maintaining the correlation between the random variables and indeed producing better generalisation for the trends.


Radial basis function (RBF) kernel (also known as Gaussian kernel) is a popularly used covariance function in GP modelling.

\begin{align}
K_{rbf}(\mathbf{x}_i, \mathbf{x}_j) &= \sigma^2 \exp\left(\frac{||\mathbf{x}_i - \mathbf{x}_j||_2^2}{2l^2}\right)\\
\end{align}

There are a variety of kernels that can be used to model different desired shapes of the fitting functions. Following parameters in the kernel function play a significant role in the modelling of a GP:
* Lengthscale ($l$)
* Variance ($\sigma^2$)

We discuss two broad categories of kernels, stationary and non-stationary in Sec. [\ref], and also compare their performances on standard datasets.
Now, let us visualize GPs applied on some standard regression datasets.


## Stationary GP on noisy sine curve dataset

In [15]:
# !pip install GPy
# !pip install chart-studio
import GPy
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
import torch
import pandas as pd
# from plotly.offline import init_notebook_mode
# init_notebook_mode(connected=False)
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [8]:
def get_fig(r=1, c=1):
    fig = make_subplots(rows=r, cols=c) 
    fig.update_layout(
        paper_bgcolor='rgb(255,255,255)',
        plot_bgcolor='rgb(255,255,255)'
    )
    return fig

In [9]:
np.random.seed(0)
X1 = np.linspace(-1, 1, 50).reshape(-1,1)
y1 = np.sin(5*X1) + np.random.rand(50, 1)*0.5
X1_ = np.linspace(-1, 1, 100).reshape(-1,1)
# X1.shape, y.shape

In [10]:
np.random.seed(0)
model = GPy.models.GPRegression(X1, y1, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y1_, y1_var = model.predict(X1_)
y1_std2 = np.sqrt(y1_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X1.ravel(), y=y1.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel()-y1_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X1_.ravel(), y=y1_.ravel()+y1_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X1.ravel().round(2), 
                         y=X1.ravel().round(2), 
                         z=model.kern.K(X1),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='$'+'\\text{Gaussian process fit on noisy sine curve dataset with RBF kernel: }l='+str(model.kern.lengthscale[0].round(2))\
    +',\;\sigma = '+str(model.kern.variance[0].round(2))+'$', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

Notice that noisy sine data is having uniform noise over the entire input region. We can also see that smoothness of sine function remains similar for any value of input $X$.

Now, we show the same model fit over a bit more comprex data.

In [11]:
np.random.seed(0)
X2 = np.linspace(-1, 1, 50).reshape(-1,1)
K2 = GPy.kern.RBF(1, lengthscale=0.5, variance=1).K(X2)
y2 = np.random.multivariate_normal(np.zeros(X2.shape[0]), K2, size=1).reshape(-1,1) + np.random.rand(50,1)
X2_ = np.linspace(-1, 1, 100).reshape(-1,1)
# X2.shape, y2.shape

In [12]:
np.random.seed(0)
model = GPy.models.GPRegression(X2, y2, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y2_, y2_var = model.predict(X2_)
y2_std2 = np.sqrt(y2_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X2.ravel(), y=y2.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X2_.ravel(), y=y2_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X2_.ravel(), y=y2_.ravel()-y2_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X2_.ravel(), y=y2_.ravel()+y2_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X2.ravel().round(2), 
                         y=X2.ravel().round(2), 
                         z=model.kern.K(X2),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='$'+'\\text{Gaussian process fit on a noisy dataset with RBF kernel: }l='+str(model.kern.lengthscale[0].round(2))\
    +',\;\sigma = '+str(model.kern.variance[0].round(2))+'$', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

There are two similarities between noisy sine curve dataset and noisy complex dataset: i) noise in data-points is uniform across $X$; ii) Underlying function that generates the dataset seems equally smooth (stationary) across $X$. 

In real word, it is completely possible that datasets may not follow one or more of the above properties. Now, we will show the performance of stationary GPs on a real-world dataset.

## Stationary GP on Olympic marathon dataset

Olympic Marathon dataset includes gold medal times for Olympic Marathon since 1896 to 2020. One of the noticable point about this dataset is that, in 1904, Marathon was badly organised leading to very slow times.  <a href="https://web.archive.org/web/20200417171704/https://www.sports-reference.com/olympics/summer/1904/ATH/mens-marathon.html">Athletics at the 1904 St. Louis Summer Games: Men's Marathon". sports-reference.com. Archived from the original on April 17, 2020. Retrieved July 22, 2017.</a>

Let us see how standard GP performs over this dataset. These fits are obtained by optimizing the likelihood function over the two important parameters: length-scale ($l$) and variance ($\sigma^2$).


In [20]:
# !pip install pods
# import pods
# data = pods.datasets.olympic_marathon_men()
data = pd.read_csv("data/olympic.csv", index_col=0)
X3 = data['X'].to_numpy().reshape(-1, 1)
y3 = data['y'].to_numpy().reshape(-1, 1)

offset = y3.mean()
scale = np.sqrt(y3.var())

y3 = (y3-offset)/scale
X3_ = X3
# X3.shape, Y3.shape

In [21]:
np.random.seed(0)
model = GPy.models.GPRegression(X3, y3, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y3_, y3_var = model.predict(X3_)
y3_std2 = np.sqrt(y3_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X3.ravel(), y=y3.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X3_.ravel(), y=y3_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X3_.ravel(), y=y3_.ravel()-y3_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X3_.ravel(), y=y3_.ravel()+y3_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X3.ravel().round(2), 
                         y=X3.ravel().round(2), 
                         z=model.kern.K(X3),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='$'+'\\text{Gaussian process fit on a Olympic dataset with RBF kernel (original 1904 observation): }l='+str(model.kern.lengthscale[0].round(2))\
    +',\;\sigma = '+str(model.kern.variance[0].round(2))+'$', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

In [22]:
# !pip install pods
# import pods
# data = pods.datasets.olympic_marathon_men()
data = pd.read_csv("data/olympic.csv", index_col=0)
X4 = data['X'].to_numpy().reshape(-1, 1)
y4 = data['y'].to_numpy().reshape(-1, 1)
y4[2] = 4.3

offset = y4.mean()
scale = np.sqrt(y4.var())

y4 = (y4-offset)/scale
X4_ = X4
# X4.shape, Y4.shape

In [23]:
np.random.seed(0)
model = GPy.models.GPRegression(X4, y4, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y4_, y4_var = model.predict(X4_)
y4_std2 = np.sqrt(y4_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X4.ravel(), y=y4.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X4_.ravel(), y=y4_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X4_.ravel(), y=y4_.ravel()-y4_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X4_.ravel(), y=y4_.ravel()+y4_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X4.ravel().round(2), 
                         y=X4.ravel().round(2), 
                         z=model.kern.K(X4),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='$'+'\\text{Gaussian process fit on a Olympic dataset with RBF kernel (adjusted 1904 observation): }l='+str(model.kern.lengthscale[0].round(2))\
    +',\;\sigma = '+str(model.kern.variance[0].round(2))+'$', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

From the above fit, we can see that data is more irregular or has higher noise till 1950, and after that, the trend in data becomes more clear and narrow. In other words, Noise in data is in decreasing order from left to right side of the plot. Predictive variance in the first fit is overestimated due to anomaly present in the year 1904. Once, we adjust the observation at 1904 with another value, the fit produces reasonable predictive variance.

If we think of an ideal fit for the original dataset, it should have decreasing predictive variance and increasing smoothness in fitted function as year increases. Standard or stationary GPs are not well-equipped internally to deal with such datasets. Such datasets are known as non-stationary and now we formally discuss stationarity and nonstationarity.


## Stationarity v/s Non-stationarity

A definition of a stationairy process from Wikipedia is as the following,

* **In mathematics and statistics, a stationary process (or a strict/strictly stationary process or strong/strongly stationary process) is a stochastic process whose unconditional joint probability distribution does not change when shifted in time.**

The definition above also applies to space or any input space. Now, let us see what does it mean in context of Gaussian processes. 

### Stationary v/s Non-stationary Gaussian Processes

##Let us plot the length scale and the variance learnt by the above model.
Fig - lscale, variance.


You might have noticed that the learnt parameters are constant for all input. This suggests that the model considers the distribution of the parameters as stationary, which might pose as a constraint for real datasets. The length-scale is an essential parameter in the RBF kernel as it is able to control the smoothness of the learnt distributions. A high length scale would mean the correlation term in RBF to be small, allowing us to get a smoother distribution. In contrast, a small length scale will enable us to capture high variance distributions of data.


Unlike stationary kernels, the non-stationary kernels are also dependent on the position of the input points along with the distance between them. So, how can we construct a kernel function in a way that can address these input-dependent variations?

- Stationary: k(x1, x2) = f(x1-x2)
- Non Stationary: k(x1, x2) = f(x1-x2, x1, x2)


### Non-stationary GPs with Local Length Scale (LLS) kernel

Now that we have built up all the necessary intuition for Non-stationary GPs, let’s take a look at a unique way of introducing nonstationarity through varying length scales. LLS GP is a two tiered framework that allows the variation of length scales over the input space. More precisely, an additional independent GP is used to model the distribution of the length scale.

The varying length scale can allow us to adjust the amount of smoothness of the function for different input positions. 

### Non-stationary GP Fit on Olympic Dataset

In [ ]:
!git clone https://github.com/patel-zeel/nsgp-torch.git
%cd nsgp-torch
!python setup.py install

In [27]:
# !pip install pods
# import pods
# data = pods.datasets.olympic_marathon_men()
# X5 = data['X']
# y5 = data['Y']
data = pd.read_csv("../data/olympic.csv", index_col=0)
X5 = data['X'].to_numpy().reshape(-1, 1)
y5 = data['y'].to_numpy().reshape(-1, 1)

offset = y5.mean()
scale = np.sqrt(y5.var())

y5 = (y5-offset)/scale
X5_ = X5
# X5.shape, y5.shape

X5 = torch.tensor(X5)
y5 = torch.tensor(y5)
X5_ = torch.tensor(X5_)

In [28]:
import torch
from nsgp import NSGP
from nsgp.utils.inducing_functions import f_kmeans

torch.manual_seed(0)

X_bar = f_kmeans(X5, num_inducing_points=4, random_state=0)
model = NSGP(X5, y5, X_bar=X_bar)

optim = torch.optim.Adam(model.parameters(), lr=0.1)

losses = []
model.train()
for _ in range(200):
    optim.zero_grad()
    loss = model.nlml()
    losses.append(loss.item())
    loss.backward()
    optim.step()

# print(losses)
model.eval()

y5_, y5_var = model.predict(X5_)
y5_ = y5_.detach().cpu()
y5_std2 = np.sqrt(y5_var.diagonal().detach().cpu())*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X5.ravel(), y=y5.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X5_.ravel(), y=y5_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X5_.ravel(), y=y5_.ravel()-y5_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X5_.ravel(), y=y5_.ravel()+y5_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X5.ravel(), 
                         y=X5.ravel(), 
                         z=model.GlobalKernel(X5, X5).detach().cpu(),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='$'+'\\text{Gaussian process fit on a Olympic dataset with RBF kernel$', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

The above fit is representative of the extra power of non stationary GPs. We observe that the fit is able to capture the outlier by having a lower length scale there. Lower length scale values are indicative of the lesser covariance between the points around the outlier. This allows the model to be more non-smooth close to the outlier, while it is able to smoothen itself on later years, when the pace timings of athletes seems to slowly converge.

Let us see some more comparisons to better understand non stationary GPs.


### Noisy Damped Sine Wave

In [ ]:
from math import sin
from math import pi
from math import exp
from matplotlib import pyplot
import random
random.seed(0)
# create sequence
length = np.linspace(0, 30, 150).tolist()
length = sorted(random.sample(length, 50))
period = 10
decay = 0.05
sequence = [(10 * sin(2 * pi * i / period) + random.gauss(0, 10))* exp(-decay * 2*i)  for i in length]
# sequence = [(100 * sin(2 * pi * i / period))* exp(-decay * 3*i)  for i in length]
# pyplot.plot(length, sequence)
# pyplot.show()
X6_d = np.array(length).reshape(-1, 1)
y6_d = np.array(sequence).reshape(-1, 1)

offset = y6_d.mean()
scale = np.sqrt(y6_d.var())

y6_d = (y6_d-offset)/scale
X6_d_ = torch.linspace(-5, 35, 100).reshape(-1, 1)
# X5.shape, y5.shape

X6_d = torch.tensor(X6_d)
y6_d = torch.tensor(y6_d)
X6_d_ = torch.tensor(X6_d_)

In [30]:
import torch
from nsgp import NSGP
from nsgp.utils.inducing_functions import f_kmeans

torch.manual_seed(0)

X_bar = f_kmeans(X6_d, num_inducing_points=3, random_state=0)
model = NSGP(X6_d, y6_d, X_bar=X_bar)

optim = torch.optim.Adam(model.parameters(), lr=0.1)

losses = []
model.train()
for _ in range(200):
    optim.zero_grad()
    loss = model.nlml()
    losses.append(loss.item())
    loss.backward()
    optim.step()

# print(losses)
model.eval()

y6_d_, y6_d_var = model.predict(X6_d_)
y6_d_ = y6_d_.detach().cpu()
y6_d_std2 = np.sqrt(y6_d_var.diagonal().detach().cpu())*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X6_d.ravel(), y=y6_d.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=1, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel()-y6_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel()+y6_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X6_d.ravel(), 
                         y=X6_d.ravel(), 
                         z=model.GlobalKernel(X6_d, X6_d).detach().cpu(),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='Non-stationary Gaussian process fit on a Damping Sine Wave dataset with RBF kernel', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()


In [31]:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x=X6_d_.ravel().flatten(), y=model.get_LS(X6_d_)[0].detach().flatten(),
                    mode= 'lines', name='Learnt Lengthscale'))
fig.update_layout(
    title_text='Learnt Length Scale: Non-stationary Gaussian process fit on a Damping Sine Wave dataset with RBF kernel', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Length-Scale",
                legend=dict(
    orientation="h", x=0, y=1.1),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(255,255,255)'
                )
fig.show()


In [32]:
np.random.seed(0)
X6_d = X6_d.numpy()
y6_d = y6_d.numpy()
X6_d_ = X6_d_.numpy()
model = GPy.models.GPRegression(X6_d, y6_d, GPy.kern.RBF(1))
model.optimize_restarts(1, verbose=0)

y6_d_, y6_d_var = model.predict(X6_d_)
y6_d_std2 = np.sqrt(y6_d_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X6_d.ravel(), y=y6_d.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel()-y6_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X6_d_.ravel(), y=y6_d_.ravel()+y6_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X6_d.ravel().round(2), 
                         y=X6_d.ravel().round(2), 
                         z=model.kern.K(X6_d),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text= 'Gaussian process fit on a Damping Sine Wave with RBF kernel: l='+str(model.kern.lengthscale[0].round(2))\
    +' sigma = '+str(model.kern.variance[0].round(2)), 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

This noise in this sine wave is introduced in proportion to the amplitude of the wave at that particular input. Hence, there is a lot more noise in the lower x positions. The NonStationary LLS GP is able to capture this high variance of the data in the lower x positions by having a lower length scale. And in the later positions it adjusts the length scale to a higher value which gets us a more smoother curve later on. However, the stationary GP, having its constraints, is not able to capture this varying variance of the data.

### Smooth 1-D Variation

In [ ]:
import random
random.seed(0)
length = np.linspace(-2, 2, 101).tolist()
# length = sorted(random.sample(length, 101))
sequence = [np.sin(i) + 2*np.exp(-30*(i)**2) +random.gauss(0,0.3) for i in length]
# plt.scatter(length, sequence)
# plt.show()
X7_d = np.array(length).reshape(-1, 1)
y7_d = np.array(sequence).reshape(-1, 1)

offset = y7_d.mean()
scale = np.sqrt(y7_d.var())

y7_d = (y7_d-offset)/scale
X7_d_ = torch.linspace(-2.5, 2.5, 100).reshape(-1, 1)
# X5.shape, y5.shape

X7_d = torch.tensor(X7_d)
y7_d = torch.tensor(y7_d)
X7_d_ = torch.tensor(X7_d_)

In [34]:
import torch
from nsgp import NSGP
from nsgp.utils.inducing_functions import f_kmeans

torch.manual_seed(15)

# X_bar = f_kmeans(X7_d, num_inducing_points=3, random_state=0)
X_bar = torch.Tensor(np.array([-1.4,0,1.4])).type(torch.DoubleTensor).reshape(-1,1)
model = NSGP(X7_d, y7_d, X_bar=X_bar)

optim = torch.optim.Adam(model.parameters(), lr=0.1)

losses = []
model.train()
for _ in range(40):
    optim.zero_grad()
    loss = model.nlml()
    losses.append(loss.item())
    loss.backward()
    optim.step()

# print(losses)
model.eval()

y7_d_, y7_d_var = model.predict(X7_d_)
y7_d_ = y7_d_.detach().cpu()
y7_d_std2 = np.sqrt(y7_d_var.diagonal().detach().cpu())*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X7_d.ravel(), y=y7_d.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=1, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel()-y7_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel()+y7_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X7_d.ravel(), 
                         y=X7_d.ravel(), 
                         z=model.GlobalKernel(X7_d, X7_d).detach().cpu(),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text='Non-stationary Gaussian process fit on a Smooth-1D dataset with RBF kernel', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

In [35]:
fig = go.Figure()
fig = fig.add_trace(go.Scatter(x=X7_d_.ravel().flatten(), y=model.get_LS(X7_d_)[0].detach().flatten(),
                    mode= 'lines', name='Learnt Lengthscale'))
fig.update_layout(
    title_text='Learnt Length Scale: Non-stationary Gaussian process fit on Smooth-1D dataset with RBF kernel', 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Length-Scale",
                legend=dict(
    orientation="h", x=0, y=1.1),
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(255,255,255)'
                )
fig.show()


In [36]:
np.random.seed(0)
X7_d = X7_d.numpy()
y7_d = y7_d.numpy()
X7_d_ = X7_d_.numpy()
model = GPy.models.GPRegression(X7_d, y7_d, GPy.kern.RBF(1))
model.optimize_restarts(5, verbose=0)

y7_d_, y7_d_var = model.predict(X7_d_)
y7_d_std2 = np.sqrt(y7_d_var)*2

fig = get_fig(1,2)
fig.update_layout(xaxis={'domain': [0, 0.5]},
        xaxis2={'domain': [0.55, 1]})
fig.add_trace(go.Scatter(x=X7_d.ravel(), y=y7_d.ravel(),
                    mode='markers',opacity=1,
                    name='data points',line=dict(width=4, color='#0078D7'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel(),
                    mode='lines',opacity=1,
                    name='predictive mean',line=dict(width=4, color='black'), 
                    hovertemplate='(%{x:.2f},%{y:.2f})'), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel()-y7_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',showlegend=False,name='Predictive variance',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode='none' # override default markers+lines
                    ), row=1, col=1)
fig.add_trace(go.Scatter(x=X7_d_.ravel(), y=y7_d_.ravel()+y7_d_std2.ravel(), fill='tonexty',
                         fillcolor='rgba(128,128,128,0.2)',
                         hovertemplate='(%{x:.2f},%{y:.2f})',
                    mode= 'none', name='Predictive variance'), row=1, col=1)

fig.add_trace(go.Heatmap(x=X7_d.ravel().round(2), 
                         y=X7_d.ravel().round(2), 
                         z=model.kern.K(X7_d),
                         colorbar=dict(x=1)), row=1, col=2)

fig.update_layout(
    title_text= 'Gaussian process fit on Smooth 1-D with RBF kernel: l='+str(model.kern.lengthscale[0].round(2))\
    +' sigma = '+str(model.kern.variance[0].round(2)), 
                title_x=0.5,
                xaxis_title="X",
                yaxis_title="Y",
                legend=dict(
    orientation="h", x=0, y=1.1)
                )

fig.show()

<a href="http://ais.informatik.uni-freiburg.de/publications/papers/plagemann08ecml.pdf"> Smooth 1-D </a>is a smoothly varying function with a substantial bump close to x = 0 . The figures here show the comparison of the stationary GP with the non stationary LLS GP. 
If we closely observe the learnt length scale figure, we can infer that the LLS GP evidently captures the existence of bump i.e, the area having lesser correlation with the other points, and has lower length scales as compared to both the sides of the bump. Although, the stationary GP here, seems to decently fit the data, we can observe that the smoothness of the trends does not remain intact near the outer regions (x < -2 and x > 2), whereas the LLS GP gives a much better generalization with help of the the learnt length scales.
